# 2. Generative Adversarial Network (GAN, 적대적 생성 네트워크)
- 두 네트워크 간의 상호 경쟁을 통해 네트워크의 성능을 향상시키는 개념을 구현한 것
- GAN은 판별자와 생성자의 두 개 네트워크로 구성
- 판별자는 합성곱 신경망, 순환 신경망 등의 일반적인 '분류 네트워크'를 사용하며, 입력받은 데이터가 원본인지 AI가 생성한 데이터인지를 분류
- 생성자는 앞 장에서 학습한 '생성 네트워크'를 사용
- 생성자는 랜덤한 숫자들을 입력으로 받아 원본과 유사한 데이터를 생성
- 여기서 랜덤한 숫자를 입력으로 받는 것은 특정한 종류의 데이터(예를 들어 특정 숫자)를 만드는 것이 목표가 아니기 때문
- 이 생성 네트워크는 어떤 숫자든지 원본 데이터와 유사한 데이터를 다양하게 만들어 내는 것을 목표로 함
- 여러 개의 랜덤한 숫자를 사용하여 다양한 입력에 따라 다양한 결과 데이터가 생성되게 하며, 학습 시에는 어떤 종류의 데이터를 생성했는지가 아닌, 원본 데이터인 것처럼 판별자를 속일 수 있는지만 고려
- 여기서 판별자는 정답을 맞힐 수 있도록 학습하지만, 생성자는 판별자가 정답을 틀리도록 학습한다는 점에 주의
1) 판별자의 학습
- 판별자의 학습을 위해 주어지는 데이터는 (1) n개의 원본 데이터와 (2) m개의 생성자에 의해 생성된 데이터
- 각 데이터 샘플의 수는 다를 수 있으나, 일반적으로는 비슷한 수준으로 유지하는 것이 학습에 용이
- 판별자는 합성곱 신경망 등의 학습과 마찬가지로, 주어진 데이터의 종류를 올바로 맞힐 수 있도록 학습됨
- 즉, 최종 출력 노드에서 발생하는 손실을 최소화하도록 학습됨
2) 생성자의 학습
- 생성자의 학습은 판별자에 비해 조금 까다로움
- 생성자 단독으로는 생성된 데이터가 원본 데이터와 유사한 특성을 가지는지 알 수 없기 때문
- 이 문제를 극복하기 위해 생성자는 판별자와 네트워크를 연결시켜서 학습
- 생성자에서 생성된 데이터는 판별자에 입력으로 주어지고, 판별자는 이 데이터가 원본으로 판단되는 정도(1이면 원본이 확실, 0이면 원본이 아님)를 출력
- 이때 판별자의 목표는 원본과 생성본을 구분해 내는 것
- 즉, 생성된 이미지가 입력되었을 때 판별자가 1에 가까운 숫자를 출력한다면 생성자 입장에서는 성공한 것이지만, 판별자는 에러를 발생시킨 것이 되며, 생성자의 학습은 판별자에서 발생하는 손실을 최대화시키는 방향으로 이루어지게 됨
- 이때 생성자가 학습하는 동안 판별자의 내부 가중치가 수정되어서는 안 된다는 점에 주의해야 함
- 판별자는 생성자와 함께 학습하며 발전해야 하는데, 생성된 이미지가 원본 데이터인 것처럼 출력하도록 판별자를 학습한다면 판별자의 성능이 하락하기 때문
- 생성자의 학습에 있어서 원본 데이터는 사용되지 않고, 생성된 데이터만 사용되는데, 이것은 생성자가 직접적으로 원본 데이터를 활용할 방법이 없기 때문
3) 배치 정규화 층
- GAN의 안정적인 구현을 위해서 '배치 정규화(Batch Normalization)층' 이라는 이름의 새로운 네트워크층이 사용됨
- 배치 정규화 층은 주어진 입력을, 평균이 0이고 표준편차가 1이 되도록 데이터를 정규화한 후 선형으로 변형하여 출력하는 네트워크층
- 여기서 선형 변형은 가중치를 곱하고, 바이어스(Bias)를 더하는 작업을 의미하며, 가중치와 바이어스는 학습 과정을 통해 조정될 수 있음
- 배치 정규화층은 신경망의 각 층에서 나오는 출력의 분포를 균일화시켜 주는 역할을 하는데, 이 작업을 통해 네트워크 학습의 속도와 성능을 향상시킬 수 있으며, 드롭아웃층의 역할을 일부 수행하여 경우에 따라서는 드롭아웃층 대신에 사용할 수도 있음
- 배치 정규화는 학습 및 테스트 도중에 이루어지므로, 전체 데이터가 아니라 배치 단위로 정규화가 이루어지며, 테스트 시에는 배치의 평균과 표준편차를 사용할 수 없으므로, 학습 단계에서의 평균 등을 사용하여 정규화 됨

## 2-1. 데이터 읽기, Reshape와 정규화
- Keras에서 사용할 수 있도록 numpy 배열 형식으로 변환
- 이미지 데이터는 28*28 크기로 변형, 0과 1 사이의 값을 갖도록 정규화
- 입력 데이터는 랜덤 벡터를 사용할 것이므로 별도로 저장하지 않음

In [ ]:
import pandas as pd
import numpy as np

d = pd.read_csv("mnist_test.csv")
d = np.array(d)
y_train = d[:, 1:].reshape(-1, 28, 28)
y_train = y_train / 255

## 2-2. 네트워크의 구성
1) 생성 네트워크의 구성
- 주요 특징은 생성 네트워크의 안정적인 학습을 위해 배치 정규화 층이 추가된 것
- 학습률은 0.002로 조정되었는데, 이것은 판별 모델에 비해 생성 네트워크가 조금 더 빠르게 학습되도록 하기 위한 것
- 판별 모델이 너무 빠르게 학습될 경우, 생성 네트워크가 그 속도를 따라가지 못하고 학습에 실패하는 경우가 발생하기 때문에, 생성 네트워크의 학습률은 데이터 종류와 상황에 따라 적절하게 조절하여 상대적으로 조금 높게 잡는 것이 좋음
- 손실함수는 binary_crossentropy가 사용되었고, 이것은 생성 네트워크의 학습이 판별 네트워크와 함께 연결되어 이루어지기 때문

In [ ]:
import tensorflow.keras as keras
from keras import Sequential
from keras import layers, optimizers

model_gen = Sequential()
model_gen.add(layers.Dense(units=3136, activation="relu"))
model_gen.add(layers.BatchNormalization())
model_gen.add(layers.Reshape((7, 7, 64)))
model_gen.add(layers.UpSampling2D((2, 2)))
model_gen.add(layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
model_gen.add(layers.BatchNormalization())
model_gen.add(layers.UpSampling2D((2, 2)))
model_gen.add(layers.Conv2D(filters=16, kernel_size=3, padding="same", activation="relu"))
model_gen.add(layers.BatchNormalization())
model_gen.add(layers.Conv2D(filters=1, kernel_size=3, padding="same", activation="sigmoid"))
model_gen.compile(loss="binary_crossentropy", optimizer=optimizers.RMSprop(learning_rate=0.002), metrics=["accuracy"])

2) 판별 모델의 구성
- 판별 모델은 합성곱층과 풀링층, Flatten층과 완전 연결층을 사용하여 구성되었고, 최종 출력 노드는 하나(0은 생성된 이미지, 1은 원본 이미지)로 설정하여, 손실함수는 binary_crossentropy가 사용됨
- 마지막 완전 연결층의 출력은 sigmoid가 사용되었는데, 이것은 최종 출력이 0과 1 사이의 실수로 나와야 하기 때문

In [ ]:
model_disc = Sequential()
model_disc.add(layers.Conv2D(filters=16, kernel_size=3, padding="same", input_shape=(28, 28, 1), activation="relu"))
model_disc.add(layers.Conv2D(filters=16, kernel_size=3, padding="same", activation="relu"))
model_disc.add(layers.MaxPooling2D(pool_size=(3,3), strides=2))
model_disc.add(layers.Conv2D(filters=16, kernel_size=3, padding="same", activation="relu"))
model_disc.add(layers.Conv2D(filters=16, kernel_size=3, padding="same", activation="relu"))
model_disc.add(layers.MaxPooling2D(pool_size=(3,3), strides=2))
model_disc.add(layers.Flatten())
model_disc.add(layers.Dense(units=1, activation="sigmoid"))
model_disc.compile(loss="binary_crossentropy", optimizer=optimizers.RMSprop(learning_rate=0.001), metrics=["accuracy"])

3) 생성 모델과 판별 모델의 연결
- 생성 모델의 학습을 위해서는 생성 모델과 판별 모델을 연결하는 모델을 만들어야 함
- Sequential 형태로 새로운 모델을 만든 다음, 생성 모델과 판별 모델을 차례로 추가하고 판별 모델의 가중치가 바뀌지 않도록 컴파일 전에 가중치를 고정
- 가중치를 고정하는 코드는 다음과 같음
- [모델].trainable = False
- 여기서 설정한 가중치의 고정은 생성 모델의 학습 시에만 적용되고, 판별 모델에서는 적용되지 않으므로, 이 값을 다시 True로 바꿀 필요는 없음

In [ ]:
model_comb = Sequential()
model_comb.add(model_gen)
model_comb.add(model_disc)
model_disc.trainable = False
model_comb.compile(loss="binary_crossentropy", optimizer=optimizers.RMSprop(learning_rate=0.001), metrics=["accuracy"])

## 2-3. 학습
- Fit 함수 하나를 호출하기만 했던 이전 네트워크들과는 달리, 생성자와 판별자를 각각 학습시켜주어야 하는 GAN에서의 학습 코드는 조금 까다로움
- GAN의 학습을 위해, 먼저 배치 크기와 입력벡터의 크기를 설정
- 배치 크기는 64로, 입력 벡터의 크기는 10으로 설정
- 이 값들은 절대적인 것이 아니므로, 상황에 따라 얼마든지 변경 가능
- 이전 네트워크들의 학습에서는 원본 이미지를 배치 크기 단위로 나누어 학습을 시키며 모든 원본 이미지를 학습에 사용하였으나, 이번 경우에는 원본 이미지와 대응하는 생성 이미지를 매 에포크마다 새롭게 생성하여야 함
- 이로 인해 전체 원본 이미지의 갯수만큼 생성하여 학습을 수행하는 것은 많은 시간이 소요되며 효율적이지 않음
- 이를 극복하기 위해, 여기에서는 원본 이미지를 배치 크기만큼 랜덤하게 선택하고 같은 갯수의 이미지를 생성한 후 해당 이미지만을 사용하여 한 번의 에포크를 완료하는 것으로 설정

In [ ]:
# 배치 크기 등 설정
nOrig = 64
nGen = nOrig
vector_size = 10
nEpoch = 2000

# 반복
for i in range(nEpoch):
  # 이미지 생성
  y_gen = np.zeros((nGen, 28, 28))
  test_input = np.random.rand(nGen, vector_size)
  for j in range(nGen):
    o = model_gen.predict(test_input[j, :].reshape(1, 10))
    o = o.reshape((28, 28))
    y_gen[j, :] = o
  y_gen = np.expand_dims(y_gen, -1)

# 원본 이미지 선택
idx = np.array(range(y_train.shape[0]))
np.random.shuffle(idx)
idx = idx[:nOrig]
y_orig = y_train[idx, :, :]
y_orig = np.expand_dims(y_orig, -1)

# 원본 이미지 - 생성 이미지 결합
test_img = np.concatenate((y_gen, y_orig), 0)
test_target = np.concatenate((np.zeros(y_gen.shape[0]), np.ones(y_gen.shape[0])), 0)

# 판별자 학습
loss_disc = model_disc.train_on_batch(test_img, test_target)

# 생성자 학습
loss_gen = model_comb.train_on_batch(test_input, np.ones(test_input.shape[0]))

- 가장 처음의 for문은 여러 에포크를 반복한다는 의미
- 각 에포크는 1) 이미지 생성, 2) 원본 이미지 선택, 3) 원본 이미지와 생성 이미지의 결합, 4) 판별자 학습, 5) 생성자 학습의 단계를 거쳐 수행됨
- 여기서 판별자와 생성자의 학습은 순차적으로 일어남
- 판별자와 생성자는 번갈아 가면서 각자의 목표를 위해 개선됨
- 판별자와 생성자는 상호간에 영향을 지속적으로 주고받으므로, 어느 한 쪽으로만 치우치지 않고, 학습의 균형을 맞추는 것이 매우 중요
- 다음은 nEpoch 만큼 학습을 반복하는 코드
- for i in range(nEpoch):

1) 이미지 생성
- 지정된 개수의 이미지를 생성하기 위해, 먼저 numpy의 rand 함수를 사용하여 입력으로 사용할 벡터를 생성
- 출력은 0으로 미리 채워진 3차원 배열로 준비
- 이 배열의 크기는 [생성할 이미지의 수] * 28 * 28이 됨
- for 문을 사용하여 반복하며 입력 벡터를 생성 네트워크에 넣어 전파시키면 해당 입력에 대한 출력을 얻을 수 있음
- 생성된 이미지 배열에는 차원을 하나 추가하여 합성곱층에서 사용할 수 있도록 함
- y_gen = np.zeros((nGen, 28, 28))
- test_input = np.random.rand(nGen, vector_size)
- for j in range(nGen):
  - o = model_gen.predict(test_input[j, :].reshape(1, 10))
  - o = o.reshape((28, 28))
  - y_gen[j, :] = o
- y_gen = np.expand_dims(y_gen, -1)
2) 원본 이미지 선택
- 판별자의 학습에 사용할 원본 이미지를 선택
- numpy.random 라이브러리의 shuffle 함수는 배열의 값을 섞는 기능을 함
- range 함수를 써서 1부터 원본 이미지 갯수까지의 정수를 일렬로 늘어놓고, shuffle 함수를 사용하여 섞은 다음, nOrig 만큼의 숫자만을 선택
- 이 숫자들이 학습에 사용할 원본 이미지의 인덱스가 됨
- 인덱스를 사용하여 이미지들을 선택하여 별도의 변수(y_orig)에 저장하고, 역시 합성곱층에서 사용할 수 있도록 차원을 추가
- idx = np.array(range(y_train.shape[0]))
- np.random.shuffle(idx)
- idx = idx[:nOrig]
- y_orig = y_train[idx, :, :]
- y_orig = np.expand_dims(y_orig, -1)
3) 원본 이미지와 생성 이미지의 결합
- 판별자를 학습하기 위해 원본 이미지와 생성 이미지를 결합하여 하나의 변수에 저장
- 각각의 이미지가 원본인지를 알려주는 레이블 정보도 함께 결합
- test_img = np.concatenate((y_gen, y_orig), 0)
- test_target = np.concatenate((np.zeros(y_gen.shape[0]), np.ones(y_gen.shape[0])), 0)
4) 판별자 학습
- 하나의 배치에 대해서만 학습을 수행하는 것이므로, 배치 단위의 학습을 위해 제공되는 train_on_batch 함수를 사용해 판별자를 학습시킴
- fit 함수가 주어진 데이터에 대해 학습을 시작부터 끝까지 수행한다면, train_on_batch 함수는 하나의 배치에 대한 학습만을 수행
- GAN에서는 하나의 배치에서 생성자와 판별자를 각각 학습시켜야 하므로, fit 함수를 사용해서는 네트워크를 학습시킬 수 없음
- train_on_batch 함수의 기본 파라미터는 '입력'과 '타깃'임
- loss_disc = model_disc.train_on_batch(test_img, test_target)
5) 생성자 학습
- 역시 train_on_batch 함수를 사용하여 생성자를 학습시킴
- 앞서 설명한 대로 생성자의 입력은 랜덤 벡터가 되며, 출력은 1에 가까워지길 기대
- 타깃 데이터는 모두 1로 만들면 되므로, numpy의 ones 함수를 사용하여 값이 모두 1인 1차원 배열을 만들어 사용
- loss_gen = model_comb.train_on_batch(test_input, np.ones(test_input.shape[0]))

## 2-4. 이미지 생성과 생성된 이미지의 확인
- 학습된 모델에 랜덤 벡터를 넣고, 이미지들을 생성
- 생성된 이미지들은 add_subplot 함수와 imshow 함수를 사용하여 화면에 표시
- add_subplot 함수는 공간을 나누어 하나의 칸 안에 여러 개의 figure가 들어가도록 하기 위한 함수며, imshow 함수는 그림을 화면에 그리는 함수
- add_subplot 함수를 사용하기 위해 figure를 하나 만들어 그림/그래프의 전체 공간 크기를 설정
- figsize의 값을 바꾸면 크기가 변경됨
- add_subplot 함수의 첫 번째, 두 번째 파라미터는 이미지가 표시될 격자를 의미
- 여기서는 8*8 크기의 격자를 정의하여 64개의 이미지가 표시되도록 함
- 세 번째 파라미터는 해당 격자에서 이미지를 표시할 위치
- for 반복문을 사용하여 i=0 부터 i=nGen-1까지 반복하므로, 위치를 i+1로 지정하여 1부터 64번 위치까지를 차례대로 가리키도록 함

In [ ]:
import matplotlib.pyplot as plt

# 10*20 크기의 figure 생성
fig = plt.figure(figsize=[20, 10])
# nGen개 랜덤 벡터의 생성
test_input = np.random.rand(nGen, vector_size)
# i=0 부터 nGen-1까지 반복
for i in range(nGen):
  # 학습된 네트워크에 랜덤 벡터를 넣어 출력 생성
  o = model_gen.predict(test_input[i, :].reshape(1, vector_size))
  # 출력의 차원을 28*28로 변경
  o = o.reshape((28, 28))
  # 8*8 격자에서 i+1번째 위치를 지정
  ax = fig.add_subplot(8, 8, i+1)
  # 출력 이미지 표시
  ax.imshow(o)

- 생성된 숫자 데이터는 평균 이미지를 만들지 않아 조금은 지저분한 감이 있지만, 실제 필기체와 더 유사한 특성을 가지고 있음
- 물론 여기서 생성된 이미지가 완벽한 것은 아님
- 글자의 형태를 온전히 갖추지 못한 경우도 제법 관찰되고 있어 개선도 필요
- 판별자와 생성자의 네트워크 구조를 조금씩 키우고, 파라미터를 조정하여 실제 데이터와 더욱 비슷한 모양의 글자를 생성